In [93]:
import bokeh
import re
import requests
import subprocess
import os

In [94]:
import pandas as pd

In [95]:
RAW_FIXTURES_DIR = "./fixtures/raw"

In [96]:
CI_pattern = '95%(.)*'
Z_pattern = 'Z-(.)*'

In [117]:
urls = {'2017':"http://www.countyhealthrankings.org/sites/default/files/state/downloads/2017%20County%20Health%20Rankings%20Indiana%20Data%20-%20v1.xls",
        '2016':"http://www.countyhealthrankings.org/sites/default/files/state/downloads/2016%20County%20Health%20Rankings%20Indiana%20Data%20-%20v3.xls",
        '2015':"http://www.countyhealthrankings.org/sites/default/files/state/downloads/2015%20County%20Health%20Rankings%20Indiana%20Data%20-%20v3.xls",
        '2014':"http://www.countyhealthrankings.org/sites/default/files/state/downloads/2014%20County%20Health%20Rankings%20Indiana%20Data%20-%20v6.xls",
       '2013':"http://www.countyhealthrankings.org/sites/default/files/state/downloads/2013%20County%20Health%20Ranking%20Indiana%20Data%20-%20v1_0.xls",
       "2012":"http://www.countyhealthrankings.org/sites/default/files/state/downloads/2012%20County%20Health%20Ranking%20Indiana%20Data%20-%20v2.xls",
       "2011": "http://www.countyhealthrankings.org/sites/default/files/state/downloads/2011%20County%20Health%20Ranking%20Indiana%20Data%20-%20v2.xls"}

In [118]:
CI_re = re.compile(CI_pattern)
Z_re = re.compile(Z_pattern)
regex_list = [CI_re, Z_re]

In [119]:
cols_to_keep=['County', 'Teen Births', 'Teen Birth Rate', 'Teen Population']

In [120]:
def get_file(url, filename):
    try:
        subprocess.call(['wget', url, '-O', filename])
    except Exception as e:
        print(filename, "failed.")
    else:
        return filename

In [121]:
def drop_bad_cols(df, regex_patterns=[]):
    bad_cols_ci = []
    for idx, col in enumerate(df.columns):
        for each_regex in regex_patterns:
            if re.search(each_regex, col):
                bad_cols_ci.append(idx)
    
    for bad_col in sorted(bad_cols_ci, reverse=True):
        df = df.drop(df.columns[bad_col], axis=1, inplace=False)

    return df

In [122]:
def keep_good_cols(df, cols_to_keep=[]):
    
    return df[cols_to_keep]

In [123]:
def load_dataset(year, url, sheetname="Ranked Measure Data", skiprows=1, bad_col_regex=list(), good_cols=list(), **kwargs):
    try:
        file_path = "{path}/teenbirth_{year}.xls".format(path = RAW_FIXTURES_DIR, year = year)
        url = url

        path_to_file = get_file(url = url, filename = file_path)
        df = pd.read_excel(path_to_file, sheetname=sheetname, skiprows=skiprows, **kwargs)
        df.columns = [col.strip() for col in df.columns]
        df = drop_bad_cols(df, regex_patterns=bad_col_regex)
        df = keep_good_cols(df, cols_to_keep = good_cols)
        df['year'] = year
    except KeyError as ke:
        print(year)
        print(df.columns)
        raise
    else:
        return df

In [124]:
dfs = {}
for year, url in urls.items():
    dfs[year] = load_dataset(year, url, bad_col_regex=regex_list, good_cols = cols_to_keep)

2011
Index(['FIPS', 'State', 'County', 'Unreliable', 'Deaths', 'YPLL Rate',
       'Sample Size', '% Fair/Poor', 'Sample Size.1',
       'Physically Unhealthy Days', 'Sample Size.2', 'Mentally Unhealthy Days',
       'Unreliable.1', 'Sample Size.3', '% LBW', 'Sample Size.4', '% Smokers',
       '% Obese', 'Sample Size.5', '% Excessive Drinking', 'Unreliable.2',
       'MV mortality rate', 'Under review', 'Cases', 'Population',
       'Rates per 100,000', 'Teen Births', 'Teen Birth Rate', '% Uninsured',
       'PCP No', 'PCP Rate', 'PCP Ratio', 'Unreliable.3', 'Population.1',
       'PCP No.1', 'PCP Rate.1', 'PCP Ratio.1', 'No of Medicare enrollees',
       'ACSC Rate', 'No of Diabetics', '% HbA1c', 'Mammography Rate',
       'Under review.1', 'AFGR', 'PSED Num', 'Population.2', 'PSED',
       '% unemployed', '% Children in Poverty', 'Sample Size.6',
       '% No Social-Emotional Support', '% Single-Parent Households',
       'Aggregate Population', 'Violent Crimes', 'Violent Crime Rate

KeyError: "['Teen Population'] not in index"

In [ ]:
master_df = pd.concat([df for df in dfs.values()])

In [111]:
master_df.to_csv("./fixtures/clean/teen_births.csv", sep = ',')

In [116]:
master_df.head

<bound method NDFrame.head of          County  Teen Births  Teen Birth Rate  Teen Population  year
0           NaN      56051.0        35.158534        1594236.0  2017
1         Adams        268.0        31.029293           8637.0  2017
2         Allen       3147.0        35.775186          87966.0  2017
3   Bartholomew        832.0        47.898676          17370.0  2017
4        Benton         62.0        31.360647           1977.0  2017
5     Blackford        132.0        46.643110           2830.0  2017
6         Boone        242.0        16.980073          14252.0  2017
7         Brown         75.0        23.098245           3247.0  2017
8       Carroll        118.0        24.471174           4822.0  2017
9          Cass        425.0        48.185941           8820.0  2017
10        Clark       1009.0        43.381057          23259.0  2017
11         Clay        247.0        38.940564           6343.0  2017
12      Clinton        349.0        46.030071           7582.0  2017
13  